이 장에서는 상속에 대해 설명하며, 특히 파이썬과 관련해서 다음 두 가지 특징을 집중적으로 알아본다.  
  
  - 내장 자료형 상속의 위험성
  - 다중 상속과 메서드 결정 순서

# 1. 내장 자료형의 상속은 까다롭다  

C 언어로 작성된 내장 클래스의 코드는 사용자가 오버라이드한 코드를 호출하지 않으므로 주의가 필요하다.

In [1]:
# __setitem__()을 오버라이드한 메서드를 무시하는 내장된 dict의 __init__()과 __update__() 메서드

class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value]*2) 

In [21]:
dd = DoppelDict(one=1) # dict 클래스의 __init__() 메서드는 __setitem__() 이 오버라이드 되었다는 사실을 무시
dd

{'one': 1}

In [22]:
dd['two']=2 # 오버라이드된 __setitem__()을 호출하므로 [2,2]에 매핑
dd

{'one': 1, 'two': [2, 2]}

In [23]:
dd.update(three=3) # update() 메서드는 오버라이드된 __setitem__() 메서드를 호출하지 않는다
dd

{'one': 1, 'two': [2, 2], 'three': 3}

이 문제는 self.get()이 self.__getitem__()을 호출하는 경우처럼 객체 안에서 호출할 때뿐만 아니라, 내장 메서드가 호출하는 다른 클래스의 오버라이드된 메서드에서도 발생한다.

In [24]:
class AnswerDict(dict):
    def __getitem__(self, key):
        return 42

In [25]:
ad = AnswerDict(a='foo') # ad는 ('a','foo') 키-값 쌍으로 채운 AnswerDict 객체
ad['a'] # __getitem__() 은 42를 반환

42

In [27]:
d = {}
d.update(ad) # update() 메서드는 AnswerDict.__getitem__() 메서드를 무시
d['a'] 

'foo'

In [28]:
d

{'a': 'foo'}

dict, list, str 등의 내장 자료형은 사용자가 정의한 오버라이드된 메서드를 무시하므로, 내장 자료형보다는 UserDict, UserList, UserString 등을 사용하는 것이 좋다.

In [29]:
# dict가 아니라 UserDict를 상속하므로 원하는 대로 작동하는 DoppleDict2와 AnswerDict2

import collections

class DoppelDict2(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value]*2)

In [30]:
dd = DoppelDict2(one=1)
dd

{'one': [1, 1]}

In [31]:
dd['two'] = 2
dd

{'one': [1, 1], 'two': [2, 2]}

In [32]:
dd.update(three=3)
dd

{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}

In [33]:
class AnswerDict2(collections.UserDict):
    def __getitem__(self, key):
        return 42

In [34]:
ad = AnswerDict2(a='foo')
ad['a']

42

In [35]:
d = {}
d.update(ad) 
d['a'] 

42

In [37]:
d

{'a': 42}

# 2. 다중 상속과 메서드 결정 순서  

다중 상속을 지원하는 언어에서는 별개의 상위클래스가 동일한 이름으로 메서드를 구현할 때 발생하는 이름 충돌 문제를 해결해야 한다.

그림 12-1
![이미지 대체 텍스트](https://pencilprogrammer.com/wp-content/uploads/2022/03/multiple-inheritance-diamond.png)

In [44]:
# 그림 12-1을 구현한 A, B, C, D 클래스의 정의

class A:
    def ping(self):
        print('ping:', self)
        
class B(A):
    def pong(self):
        print('pong:', self)

class C(A):
    def pong(self):
        print('PONG:', self)

class D(B, C):
    def ping(self):
        super().ping()
        print('post-ping:', self)
        
    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)

In [45]:
d = D()
d.pong()

pong: <__main__.D object at 0x00000146BFC30C70>


In [57]:
C.pong(d)

PONG: <__main__.D object at 0x00000146BFC30C70>


파이썬이 상속 그래프를 조회할 때는 특정한 순서를 따르므로, d.pong()과 같은 호출의 모호함이 해결된다.  

이 순서를 메서드 결정 순서(MRO)라고 한다.

In [59]:
D.__mro__


(__main__.D, __main__.B, __main__.C, __main__.A, object)

슈퍼클래스 메서드에 위임할 때는 내장 함수인 super()를 사용하는 것이 좋다. 그러나 MRO를 우회해서 슈퍼클래스 메서드를 직접 호출할 수도 있다. 예를 들어 D.ping() 메서드는 다음과 같이 구현할 수 있다.

In [60]:
def ping(self):
        A.ping(self) # super().ping() 대신 호출
        print('post-ping:', self)

객체 메서드를 클래스에 직접 호출할 때는 바인딩되지 않은 메서드에 접근하는 것이므로 self를 반드시 명시해야 한다.

In [62]:
d.ping()

ping: <__main__.D object at 0x00000146BFC30C70>
post-ping: <__main__.D object at 0x00000146BFC30C70>


In [65]:
# pingpong()이 수행한 다섯 번의 호출
d.pingpong()

ping: <__main__.D object at 0x00000146BFC30C70>
post-ping: <__main__.D object at 0x00000146BFC30C70>
ping: <__main__.D object at 0x00000146BFC30C70>
pong: <__main__.D object at 0x00000146BFC30C70>
pong: <__main__.D object at 0x00000146BFC30C70>
PONG: <__main__.D object at 0x00000146BFC30C70>


여러 클래스의 __mro __속성 조사

In [67]:
bool.__mro__

(bool, int, object)

In [68]:
def print_mro(cls):
    print(', '.join(c.__name__ for c in cls.__mro__))

In [69]:
print_mro(bool)

bool, int, object


In [82]:
import numbers
print_mro(numbers.Integral)

Integral, Rational, Real, Complex, Number, object


In [87]:
import io 

print_mro(io.BytesIO)

BytesIO, _BufferedIOBase, _IOBase, object


In [88]:
print_mro(io.TextIOWrapper)

TextIOWrapper, _TextIOBase, _IOBase, object


Tinker Texxt 위젯 클래스와 슈퍼클래스 및 Text.__mro __를 보여주는 점선 화살표
![이미지 대체 텍스트](https://picx.zhimg.com/v2-8744745813afc7584b0814e4f97996be_1440w.jpg?source=172ae18b)

# 3. 실세계에서의 다중 상속


![이미지 대체 텍스트](https://velog.velcdn.com/images/qsdcfd/post/68140d29-c0fd-46d3-9ee6-6df7974e9c99/image.png)

In [91]:
import tkinter
print_mro(tkinter.Toplevel)

Toplevel, BaseWidget, Misc, Wm, object


In [92]:
print_mro(tkinter.Widget)

Widget, BaseWidget, Misc, Pack, Place, Grid, object


In [93]:
print_mro(tkinter.Button)

Button, Widget, BaseWidget, Misc, Pack, Place, Grid, object


In [94]:
print_mro(tkinter.Entry)

Entry, Widget, BaseWidget, Misc, Pack, Place, Grid, XView, object


In [95]:
print_mro(tkinter.Text)

Text, Widget, BaseWidget, Misc, Pack, Place, Grid, XView, YView, object


# 4. 다중 상속 다루기

 - 1. 인터페이스 상속과 구현 상속을 구분한다
 - 2. ABC를 이용해서 인터페이스를 명확히 한다.
 - 3. 코드를 재사용하기 위해 믹스인을 사용한다.
 - 4. 이름을 통해 믹스인임을 명확히 한다.
 - 5. ABC가 믹스인이 될 수는 있지만, 믹스인이라고 해서 ABC인 것은 아니다.
 - 6. 두개 이상의 구상 클래스에서 상속받지 않는다.
 - 7. 사용자에게 집합 클래스를 제공한다.
 - 8. 클래스 상속보다 객체 구성을 사용하라.